In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
#importing the used classes
import ROOT
from array import array
from ROOT import TFile, TCanvas, TTree, TH1D, TH2D, gROOT, AddressOf, addressof, TGraph, TArrayD
from ROOT import TGraphErrors, TF1
from ROOT import TTreeReader, TTreeReaderArray
#defining the data structure of the file
ROOT.gInterpreter.Declare("""
struct slimport_data_t {\
    ULong64_t timetag;\
    UInt_t baseline;\
    UShort_t qshort;\
    UShort_t qlong;\
    UShort_t pur;\
    UShort_t scope[4096];\
};
""")

ROOT.gInterpreter.Declare("""
TH1D* getHistoForChannelFromTree(const char *name_file, short chan, int numBins, double minX, double maxX) {
    // variables
    slimport_data_t indata;
    TFile *infile = new TFile(name_file);
    TTree *intree = (TTree*)infile->Get("acq_tree_0");
    TBranch *inbranch = intree->GetBranch(Form("acq_ch%d",chan));
    inbranch->SetAddress(&indata.timetag);
    TH1D *h_spectrum = new TH1D("h_spectrum","Total spectrum",numBins,minX,maxX);
    // histogram filling
    for (int i=0; i<inbranch->GetEntries(); i++) {
        inbranch->GetEntry(i);
        h_spectrum->Fill(indata.qlong);
    }
    // return
    return h_spectrum;
}
""")


ROOT.gInterpreter.Declare("""
TH1D* getSummedHistoFromTree(const char *name_file, int numBins, double minX, double maxX, double m0=1, double q0=0, double m1=1, double q1=0, double m2=1, double q2=0) {
    // variables
    slimport_data_t indata0;
    slimport_data_t indata1;
    slimport_data_t indata2;
    TFile *infile = new TFile(name_file);
    TTree *intree = (TTree*)infile->Get("acq_tree_0");
    TBranch *inbranch0 = intree->GetBranch(Form("acq_ch0"));
    inbranch0->SetAddress(&indata0.timetag);
    TBranch *inbranch1 = intree->GetBranch(Form("acq_ch1"));
    inbranch1->SetAddress(&indata1.timetag);
    TBranch *inbranch2 = intree->GetBranch(Form("acq_ch2"));
    inbranch2->SetAddress(&indata2.timetag);
    TH1D *h_spectrum = new TH1D("h_spectrum","Total spectrum",numBins,minX,maxX);
    double Entry;
    // histogram filling
    for (int i=0; i<inbranch0->GetEntries(); i++) {
        inbranch0->GetEntry(i);
        inbranch1->GetEntry(i);
        inbranch2->GetEntry(i);
        Entry = indata0.qlong*m0+indata1.qlong*m1+indata2.qlong*m2+q0+q1+q2;
        h_spectrum->Fill(Entry);
    }
    // return
    return h_spectrum;
}
""")

ROOT.gInterpreter.Declare("""
TArrayD* GetDataFromTree(const char *name_file, short chan) {
    // variables
    slimport_data_t indata;
    TFile *infile = new TFile(name_file);
    TTree *intree = (TTree*)infile->Get("acq_tree_0");
    TBranch *inbranch = intree->GetBranch(Form("acq_ch%d",chan));
    inbranch->SetAddress(&indata.timetag);
    TArrayD *DataArray = new TArrayD(inbranch->GetEntries());
    // histogram filling
    for (int i=0; i<inbranch->GetEntries(); i++) {
        inbranch->GetEntry(i);
        DataArray->AddAt(indata.qlong,i);
    }
    // return
    return DataArray;
}
""")


from ROOT import slimport_data_t, getHistoForChannelFromTree, getSummedHistoFromTree, GetDataFromTree

input_line_170:2:8: error: redefinition of 'slimport_data_t'
struct slimport_data_t {    ULong64_t timetag;    UInt_t baseline;    US...
       ^
input_line_63:2:8: note: previous definition is here
struct slimport_data_t {    ULong64_t timetag;    UInt_t baseline;    US...
       ^
input_line_171:2:7: error: redefinition of 'getHistoForChannelFromTree'
TH1D* getHistoForChannelFromTree(const char *name_file, short chan, int ...
      ^
input_line_64:2:7: note: previous definition is here
TH1D* getHistoForChannelFromTree(const char *name_file, short chan, int ...
      ^
input_line_173:2:10: error: redefinition of 'GetDataFromTree'
TArrayD* GetDataFromTree(const char *name_file, short chan) {
         ^
input_line_66:2:10: note: previous definition is here
TArrayD* GetDataFromTree(const char *name_file, short chan) {
         ^


In [44]:
def GetHisto(file, channel=0, nbins=4000, xmin=0, xmax=30000):
    histo = TH1D
    histo = getHistoForChannelFromTree(file, channel, nbins, xmin, xmax)

    return histo; 

def GetSummedHisto(file, nbins=4000, xmin=0, xmax=30000,m0=1,q0=0,m1=1,q1=0,m2=1,q2=0):
    histo = TH1D
    histo = getSummedHistoFromTree(file, nbins, xmin, xmax,m0,q0,m1,q1,m2,q2)

    return histo; 


def Find_calib_par(centroids, sigmas, energies, Title="Calibration"):
    c_cal = TCanvas("Calibration_canvas","Calibar graphs ")
    c_cal.SetGrid()
    m = 1.
    q = 0.
    
    GraphErr = TGraphErrors(len(centroids),centroids,energies,sigmas)
    
    #GraphErr.SetMarkerStyle(22)
    GraphErr.SetLineColor(2)
    XAxis=GraphErr.GetXaxis()
    XAxis.SetTitle("Bins [A.U]")
    XAxis.SetTitleSize(0.037)
    XAxis.CenterTitle()
    YAxis=GraphErr.GetYaxis()
    YAxis.SetTitle("Energy [keV]")
    YAxis.SetTitleSize(0.037)
    YAxis.CenterTitle()
    GraphErr.SetTitle(Title)

    fitfunc = TF1("calfitfun1","pol1",np.amin(centroids),np.amax(centroids))
    fitfunc.SetParameters(0,0.1)
    
    GraphErr.Fit(fitfunc,"R")
    GraphErr.Draw()
    
    m = fitfunc.GetParameter(1)
    q = fitfunc.GetParameter(0)
    
    print("Slope = %f, Intercept = %f" %(m,q))
    
    c_cal.GraphErr = GraphErr
    c_cal.Draw()
    return m,q,c_cal

def CalibrateHisto(h_uncal, m, q):
    
    max_bin = h_uncal.GetNbinsX() #This method returns the number of bins in x of the histogram
    max_kev = h_uncal.GetBinCenter(max_bin)*m + q
    XAxis = h_uncal.GetXaxis()
    XAxis.SetLimits(q,max_kev)
    if (m!=1 and q!=0): #This means that I actually changed the calibration!
        h_uncal.SetXTitle("keV")
    Cal_histo = h_uncal.Clone()
    
    return Cal_histo

def CalibrateTimeHisto(h_uncal, m, q):
    
    max_bin = h_uncal.GetNbinsX() #This method returns the number of bins in x of the histogram
    max_t = h_uncal.GetBinCenter(max_bin)*m + q
    XAxis = h_uncal.GetXaxis()
    XAxis.SetLimits(q,max_t)
    if (m!=1 and q!=0): #This means that I actually changed the calibration!
        h_uncal.SetXTitle("[ns]")
    Cal_histo = h_uncal.Clone()
    
    return Cal_histo


def subtract_bg(Histo, Histo_time, BG, BG_time):
    
    h_subtr = Histo.Clone()
    h_subtr.Add(BG,-Histo_time/BG_time)
    
    return h_subtr;




def peaksearch(Histo, xmin, xmax, ratio):
    
    h_peaks = Histo.Clone()
    XAxis = h_peaks.GetXaxis()
    XAxis.SetRangeUser(xmin, xmax)
    
    Histo.SetTitle("Measured spectrum")
    h_peaks.SetTitle("Peak search")
    
    s = ROOT.TSpectrum(30)
    
    Sigma =10
    
    nPeaks = s.Search(h_peaks,Sigma,"",ratio)
    xPeaks = s.GetPositionX()
    
    buffer=[]
    for p in range(nPeaks):
        print("Peak #%d @ channel %f" %(p,xPeaks[p]))
        buffer = np.append(buffer,xPeaks[p])
    xPeaks=buffer  
        
    return nPeaks, xPeaks


def Sub_BKG(Histo, SubComp, Binning , nI):
    nbins=Histo.GetNbinsX()
    xmin=0
    xmax=30000
    gROOT.ForceStyle()
    
    d1= TH1D("d1","", nbins, xmin, xmax)
    
    #Histo.SetTitle("Estimation of background with Compton edges under peaks");
    #XAxis=Histo.GetXaxis()
    #XAxis.SetRange(1,nbins)
    Histo.Draw("L")
    
    s = ROOT.TSpectrum()
    source = np.empty(nbins)
    if SubComp==True:
        for i in range(nbins):
            source[i] = Histo.GetBinContent(i + 1)
        s.Background(source,nbins,nI,1,3,True,3,True)
    else:
        for i in range(nbins):
            source[i] = Histo.GetBinContent(i + 1)
        s.Background(source,nbins,nI,1,3,True,3,True)
        
    for i in range(nbins):
        d1.SetBinContent(i + 1,source[i])
        
    h_subtr = Histo.Clone()
    h_subtr.Add(d1,-1)
    
    return h_subtr

In [18]:
def Auto_calibration_Na22(Histo):
    H_Subtr=Sub_BKG(Histo,True,10,120)
    nPeaks,xPeaks = peaksearch(H_Subtr,1000,20000,0.15)
    
    Gf=[]
    Gpar=[]
    Function = ""
    for i in range(nPeaks):
        g = TF1("g%d"%i, "gaus", xPeaks[i]-150, xPeaks[i]+150)
        Gf.append(g)
        if i==0:
            H_Subtr.Fit(Gf[i],"QNR")
            Function = Function + ("gaus(%d)"%(i*3))
        else:
            H_Subtr.Fit(Gf[i],"QNR+")
            Function = Function + ("+gaus(%d)"%(i*3))
        Par=Gf[i].GetParameters()
        for k in range(3):
            Gpar = np.append(Gpar,Par[k])
            
    F_tot = TF1("F_tot",Function,0,30000)
    F_tot.SetParameters(Gpar)
    H_Subtr.Fit(F_tot,"QR+")
    Gpar = F_tot.GetParameters()
    
    Centroids = np.array([Gpar[1],Gpar[4]])
    Sigma = np.array([Gpar[2],Gpar[5]])
    Energies = np.array([511.,1274.5])
    
    m,q,calic_canvas = Find_calib_par(Centroids,Sigma,Energies)
    
    return m,q
    
    
    

# Detector Energy Calibration

In [45]:
i=1
print("Detector %d"%i)
i+=1
C=TCanvas()
#Histo0=GetHisto("Dati_calib/detector1_calibrazione10min.root", 1, 3000, 0, 30000)
Histo0=GetHisto("Group5/3_giornata/caldetector1_giusta.root", 0, 3000, 0, 30000)
m0,q0=Auto_calibration_Na22(Histo0)
print("Detector %d"%i)
i+=1
Histo1=GetHisto("Group5/1_giornata/detector2_calibrazione10min.root", 1, 3000, 0, 30000)
m1,q1=Auto_calibration_Na22(Histo1)
print("Detector %d"%i)
i+=1
Histo2=GetHisto("Group5/1_giornata/detector3_calibrazione10min.root", 1, 3000, 0, 30000)
m2,q2=Auto_calibration_Na22(Histo2)
print("Detector %d"%i)
i+=1
Histo3=GetHisto("Group5/1_giornata/detector4_calibrazione_5min.root", 1, 3000, 0, 30000)
m3,q3=Auto_calibration_Na22(Histo3)

Canvas = TCanvas()
Histo0.Draw()
Canvas.Draw()


Detector 1
Peak #0 @ channel 4695.000000
Peak #1 @ channel 11465.000000
Slope = 0.112786, Intercept = -18.665405
Detector 2
Peak #0 @ channel 4685.000000
Peak #1 @ channel 11355.000000
Slope = 0.114449, Intercept = -23.565128
Detector 3
Peak #0 @ channel 5135.000000
Peak #1 @ channel 12445.000000
Slope = 0.104643, Intercept = -25.869384
Detector 4
Peak #0 @ channel 3195.000000
Peak #1 @ channel 7755.000000
Slope = 0.167686, Intercept = -23.556055
 FCN=3.84366e-10 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=7.6851e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.86654e+01   3.56005e+01   7.26473e-03   2.93019e-07
   2  p1           1.12786e-01   5.00710e-03   1.02187e-06   9.67398e-03
 FCN=1.28579e-09 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                  

# TAC Calibration

In [46]:
ROOT.enableJSVis()
Histo0ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_0ns_range200ns_multi10.root", 3, 500, 5000, 25000)
Histo4ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_4ns_range200ns_multi10.root", 3, 500, 5000, 25000)
Histo8ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_8ns_range200ns_multi10.root", 3, 500, 5000, 25000)
Histo16ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_16ns_range200ns_multi10.root", 3, 500, 5000, 25000)
Histo32ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_32ns_range200ns_multi10.root", 3, 500, 5000, 25000)
Histo48ns=GetHisto("Group5/3_giornata/cal_giusti/TACcal_48ns_range200ns_multi10.root", 3, 500, 5000, 25000)

TACHistos=[Histo0ns,Histo4ns,Histo8ns,Histo16ns,Histo32ns,Histo48ns]
time_arr=np.array([0.0,4.0,8.0,16.0,32.0,48.0])
centroids=[]
sigmas=[]

for i in range(len(time_arr)):
    nPeaks,xPeaks = peaksearch(TACHistos[i],5000,25000,0.5)
    gT = TF1('gT','gaus',xPeaks[0]-500,xPeaks[0]+500)
    TACHistos[i].Fit(gT,'R')
    mean=gT.GetParameter(1)
    sigma=gT.GetParameter(2)
    centroids=np.append(centroids,mean)
    sigmas=np.append(sigmas,sigma)
    
centroids=np.array(centroids)
sigmas=np.array(sigmas)


TimeGraph=TGraphErrors(len(centroids),centroids,time_arr,sigmas)
fitfunc = TF1("fitfun","pol1",np.amin(centroids),np.amax(centroids))
fitfunc.SetParameters(0,1)
    
TimeGraph.Fit(fitfunc,"R")
CTE=TCanvas()
TimeGraph.Draw()
CTE.Draw()
    
mT = fitfunc.GetParameter(1)
qT = fitfunc.GetParameter(0)

print("Slope = %f, Intercept = %f" %(mT,qT))


Peak #0 @ channel 12820.000000
Peak #0 @ channel 13140.000000
Peak #0 @ channel 13500.000000
Peak #0 @ channel 14180.000000
Peak #0 @ channel 15580.000000
Peak #0 @ channel 16940.000000
Slope = 0.011621, Intercept = -148.769592
 FCN=48.6131 FROM MIGRAD    STATUS=CONVERGED      65 CALLS          66 TOTAL
                     EDM=1.49529e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.47472e+02   7.20906e+00   1.89461e-02  -2.18286e-05
   2  Mean         1.28001e+04   3.03038e+00   1.03175e-02   1.95847e-05
   3  Sigma        1.83954e+02   2.60785e+00   1.24758e-05   3.33191e-03
 FCN=52.7297 FROM MIGRAD    STATUS=CONVERGED      63 CALLS          64 TOTAL
                     EDM=1.24247e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE

# 3 Photons decay

In [47]:
#Get the data from file and convert to np array
D1Data_TArray=GetDataFromTree("Group5/3_giornata/3fotoni20oremulti10range200ns.root",0)
D1Data = np.empty(D1Data_TArray.GetSize())
D2Data_TArray=GetDataFromTree("Group5/3_giornata/3fotoni20oremulti10range200ns.root",1)
D2Data = np.empty(D2Data_TArray.GetSize())
D3Data_TArray=GetDataFromTree("Group5/3_giornata/3fotoni20oremulti10range200ns.root",2)
D3Data = np.empty(D3Data_TArray.GetSize())
TACData_TArray=GetDataFromTree("Group5/3_giornata/3fotoni20oremulti10range200ns.root",3)
TACData = np.empty(TACData_TArray.GetSize())

#convert to numpy array
for i in range(D1Data_TArray.GetSize()):
    D1Data[i] = D1Data_TArray.GetAt(i)

for i in range(D2Data_TArray.GetSize()):
    D2Data[i] = D2Data_TArray.GetAt(i)
    
for i in range(D3Data_TArray.GetSize()):
    D3Data[i] = D3Data_TArray.GetAt(i)
    
for i in range(TACData_TArray.GetSize()):
    TACData[i] = TACData_TArray.GetAt(i)
    
#Data calibration
D1Data = D1Data*m0+q0
D2Data = D2Data*m1+q1
D3Data = D3Data*m2+q2
TACData = TACData*mT+qT


testHisto = TH1D('testHisto','',800,0,1400)
for data in D3Data:
    testHisto.Fill(data)
    
g1 = TF1('g1','gaus',270,420)
testHisto.Fit(g1,'R')

Ct = TCanvas()
testHisto.Draw()
Ct.Draw()

 FCN=149.553 FROM MIGRAD    STATUS=CONVERGED      78 CALLS          79 TOTAL
                     EDM=1.06914e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.25409e+02   3.59840e+00   1.57150e-02   3.69431e-04
   2  Mean         3.32994e+02   4.60658e-01   2.52747e-03  -6.12516e-04
   3  Sigma        5.06607e+01   5.62324e-01   1.80519e-05   4.10614e-01


In [48]:
#Sum the 3 energy and discriminate the wrong ones
disc_high=540
disc_low=180
Index1p = np.where(D1Data>=disc_high)
Index1l = np.where(D1Data<=disc_low)
Index2p = np.where(D2Data>=disc_high)
Index2l = np.where(D2Data<=disc_low)
Index3p = np.where(D3Data>=disc_high)
Index3l = np.where(D3Data<=disc_low)
Index=np.hstack((Index1p,Index1l,Index2p,Index2l,Index3p,Index3l))
Index=np.unique(Index)
D1Data=np.delete(D1Data,Index)
D2Data=np.delete(D2Data,Index)
D3Data=np.delete(D3Data,Index)
TACData=np.delete(TACData,Index)
Somma=np.add(D1Data,D2Data)
Somma=np.add(Somma,D3Data)

SummedHisto = TH1D('SummedHisto','',300,500,1550)
for data in Somma:
    SummedHisto.Fill(data)
    
SummedHisto.SetTitle('Summed energy Histogram')
    
SummedHisto_Final=Sub_BKG(SummedHisto,True,1050/300,65)
SummedHisto_Final.SetLineColor(3)
    
    
g1 = TF1('g1','gaus',910,1120)
SummedHisto_Final.Fit(g1,'R')
    
CS = TCanvas()
SummedHisto.Draw()
SummedHisto_Final.Draw('SAME')
CS.Draw()

 FCN=285.878 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=3.8785e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.43225e+02   6.32844e+00   4.27159e-02   4.04161e-06
   2  Mean         1.02280e+03   2.77960e-01   2.27425e-03  -6.76345e-05
   3  Sigma        2.32782e+01   1.97029e-01   1.77591e-05   4.59784e-03


Warning in <TH1D::Add>: Attempt to add histograms with different axis limits


In [49]:
TACHisto = TH1D('TACHisto','',70,-70,200)
for data in TACData:
    TACHisto.Fill(data)

TACHisto.SetTitle('Time distribution')
    
CTAC = TCanvas()
TACHisto.Draw()
CTAC.Draw()

# 2 Photons decay

In [51]:
#Get the data from file and convert to np array
D1Data_TArray=GetDataFromTree("Group5/3_giornata/2fotoni48ore_46sec_multi10range200ns.root",0)
D1Data = np.empty(D1Data_TArray.GetSize())
D2Data_TArray=GetDataFromTree("Group5/3_giornata/2fotoni48ore_46sec_multi10range200ns.root",1)
D2Data = np.empty(D2Data_TArray.GetSize())
TACData_TArray=GetDataFromTree("Group5/3_giornata/2fotoni48ore_46sec_multi10range200ns.root",3)
TACData = np.empty(TACData_TArray.GetSize())

#convert to numpy array
for i in range(D1Data_TArray.GetSize()):
    D1Data[i] = D1Data_TArray.GetAt(i)

for i in range(D2Data_TArray.GetSize()):
    D2Data[i] = D2Data_TArray.GetAt(i)
    
for i in range(TACData_TArray.GetSize()):
    TACData[i] = TACData_TArray.GetAt(i)
    
#Data calibration
D1Data = D1Data*m0+q0
D2Data = D2Data*m1+q1
TACData = TACData*mT+qT

testHisto = TH1D('testHisto','',800,0,1400)
for data in D1Data:
    testHisto.Fill(data)
    
Ct = TCanvas()
testHisto.Draw()
Ct.Draw()

In [53]:
#Sum the 2 energy and discriminate the wrong ones
disc_high=600
disc_low=350
Index1p = np.where(D1Data>=disc_high)
Index1l = np.where(D1Data<=disc_low)
Index2p = np.where(D2Data>=disc_high)
Index2l = np.where(D2Data<=disc_low)
Index=np.hstack((Index1p,Index1l,Index2p,Index2l))
Index=np.unique(Index)
D1Data=np.delete(D1Data,Index)
D2Data=np.delete(D2Data,Index)
TACData=np.delete(TACData,np.delete(Index,np.where(Index>412480)))

Somma=np.add(D1Data,D2Data)

SummedHisto = TH1D('SummedHisto','',800,500,1500)
for data in Somma:
    SummedHisto.Fill(data)

TACHisto = TH1D('TACHisto','',100,-50,50)    
for data in TACData:
    TACHisto.Fill(data)
    
SummedHisto.SetTitle('Summed energy Histogram')
    
    
g1 = TF1('g1','gaus',980,1110)
SummedHisto.Fit(g1,'R')
    
CS = TCanvas()
SummedHisto.Draw()
CS.Draw()

TACHisto.SetTitle('Time distribution')
    
CTAC = TCanvas()
TACHisto.Draw()
CTAC.Draw()

 FCN=3266.99 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=6.54173e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.09962e+05   5.99390e+01   1.32342e+00   1.87788e-07
   2  Mean         1.03186e+03   1.17288e-02   4.92028e-04  -9.07017e-02
   3  Sigma        2.48743e+01   9.40144e-03   2.79822e-06   1.82204e+00


Warning in <TFile::Append>: Replacing existing TH1: SummedHisto (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: TACHisto (Potential memory leak).


In [55]:
#Get the data from file and convert to np array
D1Data_TArray=GetDataFromTree("Group5/3_giornata/2fotoni1oramulti10range200ns_tempomorto.root",0)
D1Data = np.empty(D1Data_TArray.GetSize())
D2Data_TArray=GetDataFromTree("Group5/3_giornata/2fotoni1oramulti10range200ns_tempomorto.root",1)
D2Data = np.empty(D2Data_TArray.GetSize())
TACData_TArray=GetDataFromTree("Group5/3_giornata/2fotoni1oramulti10range200ns_tempomorto.root",3)
TACData = np.empty(TACData_TArray.GetSize())

#convert to numpy array
for i in range(D1Data_TArray.GetSize()):
    D1Data[i] = D1Data_TArray.GetAt(i)

for i in range(D2Data_TArray.GetSize()):
    D2Data[i] = D2Data_TArray.GetAt(i)
    
for i in range(TACData_TArray.GetSize()):
    TACData[i] = TACData_TArray.GetAt(i)
    
#Data calibration
D1Data = D1Data*m0+q0
D2Data = D2Data*m1+q1
TACData = TACData*mT+qT

testHisto = TH1D('testHisto','',800,0,1400)
for data in D1Data:
    testHisto.Fill(data)
    
Ct = TCanvas()
testHisto.Draw()
Ct.Draw()

In [56]:
#Sum the 2 energy and discriminate the wrong ones
disc_high=600
disc_low=370
Index1p = np.where(D1Data>=disc_high)
Index1l = np.where(D1Data<=disc_low)
Index2p = np.where(D2Data>=disc_high)
Index2l = np.where(D2Data<=disc_low)
Index=np.hstack((Index1p,Index1l,Index2p,Index2l))
Index=np.unique(Index)
D1Data=np.delete(D1Data,Index)
D2Data=np.delete(D2Data,Index)
TACData=np.delete(TACData,np.delete(Index,np.where(Index>412480)))

Somma=np.add(D1Data,D2Data)

SummedHisto = TH1D('SummedHisto','',800,500,1500)
for data in Somma:
    SummedHisto.Fill(data)
    
SummedHisto.SetTitle('Summed energy Histogram')

TACHisto = TH1D('TACHisto','',100,-50,50)    
for data in TACData:
    TACHisto.Fill(data)
    
TACHisto.SetTitle('Time distribution')
    
    
g1 = TF1('g1','gaus',980,1110)
SummedHisto.Fit(g1,'R')
    
CS = TCanvas()
SummedHisto.Draw()
CS.Draw()
    
CTAC = TCanvas()
TACHisto.Draw()
CTAC.Draw()

 FCN=172.153 FROM MIGRAD    STATUS=CONVERGED      71 CALLS          72 TOTAL
                     EDM=6.1255e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.60157e+03   7.20128e+00   3.69901e-02  -1.08617e-05
   2  Mean         1.03159e+03   9.68728e-02   5.98302e-04   8.85117e-04
   3  Sigma        2.45512e+01   7.49368e-02   5.22040e-06  -5.36635e-02


In [ ]:
SummedHisto.Integral(200,400)

In [ ]:
TACHisto.Integral(10,60)